#  Jogo variante do puzzle 2084

## Introdução à Inteligência Artificial edição 2021/22
### Projeto nº 2

<img src=".\imagens_2048\1024s.PNG" alt="Drawing" style="width: 200px;"/>

### Grupo: 48

#### Elementos do Grupo

Nome: Afonso Esteves

Número: 54394

Nome: João Anjos

Número: 54476

Nome: Vicente Sousa

Número: 55386

## Introdução
O objetivo deste projeto era realizar uma implementação duma variante do jogo 2048, assim como pelo menos dois jogadores que o fossem capaz de jogar, utilizando o algoritmo minimax variante alfa-beta com profundidade limitada.  
<br>
O jogo 2048 corresponde a um puzzle, jogado num tabuleiro 4x4, ou seja, com 16 posições ou "células", que exige que se combinem os números nas células (múltiplos de 2, tendo o estado inicial do tabuleiro duas posições ocupadas com este valor),combinação esta que é efetuada através do deslizamento das peças para uma de quatro direções (cima, baixo, esquerda e direita), de forma a chegar a 2048 pontos, sendo que os pontos são obtidos através da soma das peças do tabuleiro. No entanto, na variante desenvolvida, esta limitação é retirada:
<br>
O jogo passa a ser jogado por dois jogadores, um atacante e um defensor, sendo que o primeiro procura obter a maior pontuação possível, deslizando e combinando as peças, e o segundo procura minimizá-la, colocando novas peças (com o valor 2) no tabuleiro, uma por jogada, sendo que jogam à vez, começando pelo atacante, e o jogo só acaba quando o atacante não puder realizar nenhuma jogada, ou seja, quando não houver nenhum par de peças adjacentes com o mesmo número ou o tabuleiro estiver sem casas vazias.

## Formulação do Jogo 2048 em termos de estados e de operadores

### Descrição da representação dos estados do jogo

A representação de estados é conseguida através da definição da classe Jogo2048State, filha de GameState. Esta classe define as funções básicas para a manipulação e avaliação de estados. 

Cada estado é descrito por um tuplo de quatro valores:
```python 
GameState = namedtuple('GameState', 'to_move, utility, board, moves')
```

* **to_move** corresponde ao jogador que irá jogar a seguir no estado atual
* **utility** indica o número de pontos obtidos no estado atual (igual para ambos os jogadores)
* **board** é a matriz 4x4 que representa o tabuleiro, assim como as peças nele colocado
* **moves** contabiliza o número de jogadas efetuadas desde o estado inicial até o estado atual

Por cada estado é possível obter as ações possíveis sobre ele mesmo, de acordo com o player, o estado resultante após aplicar uma ação sobre o mesmo e várias outras funções de auxílio como o other que retorna o jogador complementar ao to_move, o display que escreve uma representação visual do tabuleiro para o standard output e o _collapse que calcula as colisões após o tabuleiro ter sido "deslizado". A implementação correspondente (```Jogo2048State```), retirada do ficheiro segue-se:

```python 
class Jogo2048State(GameState):
    
    """Returns a new state representing the board after the attacker player chooses a direction"""
    def __collapse(self, direction):
        try:
            output = actions[direction](self.board)
            newstate = Jogo2048State(to_move="defensor", utility = self.utility + output[1], board = output[0], moves=self.moves+1)
        except KeyError:
            raise RuntimeError("Error - invalid direction of movement" )

        return newstate
    
    """Returns a new state representing the board after a player action, doesn't check whether the action is valid or not"""
    def next_move(self, move):
        if self.to_move == "atacante":
            return self.__collapse(move)
        elif self.to_move == "defensor":
            newstate = Jogo2048State(to_move="atacante", utility = self.utility, board = copy.deepcopy(self.board), moves=self.moves+1)
            newstate.board[int(move[0])][int(move[2])] = 2
            return newstate
        else:
            raise RuntimeError("Error - invalid player descriptor")

    """Returns the other player, the one not playing this turn"""
    def other(self):
        if self.to_move == "atacante":
            return "defensor"
        if self.to_move == "defensor":
            return "atacante"
        else:
            raise RuntimeError("Error - invalid player descriptor")

    """Prints the board"""
    def display(self):
        print("="*28)
        for i in self.board:
            for j in i:
                print(alignLeft(str(j), 5), end=" ")
            print()
        print("="*28)

    """Returns all valid moves for the state"""
    def get_moves(self):
        if self.to_move == "atacante":
            return [ a for a in ["cima", "esquerda", "direita", "baixo"] if self.__collapse(a).board != self.board ]
        if self.to_move == "defensor":
            res = []
            for i in range(4):
                for j in range(4):
                    if self.board[i][j] == 0:
                        res.append(str(i)+","+str(j))
            return res
        else:
            raise RuntimeError("Error - invalid player descriptor")
```

### Testes da formulação

#### Situações iniciais dos jogos
Uso do construtor e "display" de jogos iniciais

Construção de um novo jogo com a situação inicial seguinte:

<img src=".\imagens_2048\inicial_2048.PNG" alt="Drawing" style="width: 150px;"/>
<p style="text-align: center;">Figura 1</p>

In [1]:
from IIA2122_proj2_48 import *
game = Jogo2048_48({(3,2), (3,3)})

Eis o display desse estado inicial do jogo:

In [2]:
game.display(game.initial)

    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 


#### As acções
Demonstração das acções possíveis e execução de acções para diversas situações que podem ser simples, por exemplo, iniciais ou mais complexas (a meio do jogo, já com muitas células preechidas.

As acções para o estado inicial do jogo na Fig. 1

In [3]:
initial_acts = game.actions(game.initial)
print(initial_acts)

['cima', 'esquerda', 'direita']


#### Execução das acções

Execute a acção direita para o estado da figura 1 e faça o seu display

In [4]:
state2 = game.result(game.initial, "direita")
game.display(state2)

    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     4 


Mostre o estado inicial outra vez, para confirmar que não se "deformou" devido à execução da acção. 

In [5]:
game.display(game.initial)

    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 


Mostre como ficaria a execução da acção na situação na figura 2

<img src=".\imagens_2048\deslizar_2048.PNG" alt="Drawing" style="width: 350px;"/>
<p style="text-align: center;">Figura 2</p>

In [6]:
fig2 = [[0,2,4,2],[2,0,8,2],[2,16,8,2],[2048,32,8,2]]
state3 = game.result(Jogo2048State(to_move = "atacante", utility= 2138, board = fig2, moves=0), "baixo")
game.display(state3)

    0     0     0     0 
    0     2     4     0 
    4    16     8     4 
 2048    32    16     4 


Mostre que ao aplicar a sequência de acções, (criada na célula a seguir) ao estado inicial da Figura 3, ficará com o jogo na situação ilustrada na Figura 4. O ideal é fazer uma função que executa uma sequência de acções.

| <p style="text-align: center;">Início</p>      | <p style="text-align: center;">Fim</p> |
| :---        |    :----:   |
| <img src="imagens_2048\seq_teste_begin.PNG" alt="Drawing" style="width: 150px;"/>      | <img src="imagens_2048\seq_teste_end.PNG" alt="Drawing" style="width: 150px;"/>       |   |
| <p style="text-align: center;">Figura 3</p>   | <p style="text-align: center;">Figura 4</p>        |

In [7]:
seqTeste=['cima',"1,0","cima","3,3","cima","2,0","direita","2,0","esquerda","1,0",
          "baixo","0,2","baixo","0,0","direita","2,0","cima","2,2","baixo","1,0",
          "esquerda","0,1","baixo","0,2","esquerda","0,2","baixo","0,3","cima","2,3",
          "cima","2,0","esquerda","3,2","cima","1,2","esquerda","3,1","direita","3,1",
          "direita","0,0"]

fig3 = [[0,0,0,0],[0,0,0,0],[0,2,2,0],[0,0,0,0]]
state4 = game.resultActions(Jogo2048State(to_move = "atacante", utility = 0, board = fig3, moves=0), seqTeste)
game.display(state4)

    2     0     0    32 
    0     0     4     2 
    0     0     0     2 
    0     0     0     4 


#### Demonstração que o teste de estado final do jogo funciona
Podem testar para a situação na Figura 4.

<img src=".\imagens_2048\the-end-2084.PNG" alt="Drawing" style="width: 150px;"/>
<p style="text-align: center;">Figura 4</p>

In [8]:
fig4 = [[16,128,32,16],[8,32,2,8],[2,4,16,2],[4,2,8,4]]
print(str(game.terminal_test(Jogo2048State(to_move="atacante",utility=0,board=fig4, moves=0))))

True


## Jogos entre jogadores simples
Nesta secção irão realizar alguns jogos, para verificar a modelização

### Jogadores aleatórios de ataque e de defesa
Mostre que a sua função que realiza jogos entre dois jogadores (com e sem timeout) funciona para dois jogadores aleatórios.  

In [20]:
#com timeout
rand = randomGame().jogarTimeout(atacante_ali.alg, defensor_ali.alg, 10, False)
#sem timeout
rand = randomGame().jogarTimeout(atacante_ali.alg, defensor_ali.alg, verbose=False)

TypeError: unhashable type: 'list'

Faça o display de um dos jogos realizados atrás

In [19]:
randomGame().jogarTimeout(atacante_ali.alg, defensor_ali.alg, 10, True)

TypeError: unhashable type: 'list'

### Jogadores de ataque e de defesa obsessivos


Crie um dois jogadores obsessivos: o atacante obsessivo, prefere sempre por esta ordem, as acções: 'cima', 'esquerda','direita' e 'baixo'. O obsessivo da defesa prefere a célula mais no topo e mais à esquerda possível.
Realize um jogo entre eles e faça o seu display.

In [18]:
game.jogar(atacante_obsessivo.alg, defensor_obsessivo.alg)

    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     2     2 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    2     0     2     2 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    4     2     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    4     2     2     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    4     4     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    4     4     2     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    8     2     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    8     2     2     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    8     4     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 


1548

## Exemplos de jogadores alfabeta
Desenvolvemos quatro principais funções de avaliação, de forma a chegar à melhor solução. Estas consistem em:

* **BoardAvg**: retorna de 0 a 1 o quão condensadas estão as peças em valores mais altos.
* **BoardEmpty**: retorna de 0 a 1 o quão vazio o tabuleiro está.
* **BoardComb**: retorna de 0 a 1 o potencial do tabuleiro em termos de combinações, isto é, peças que se deslizadas combinam umas com as outras.
* **BoardPos**: retorna de 0 a 1 o quão favorável é a distribuição de peças pelo tabuleiro, favorecendo tabuleiros com as peças mais altas em um dos cantos.

Os testes executados divedem-se em dois:
* Teste dos casos limites, onde o critério deve ter valor total ou nulo segundo um tabuleiro especialmente desenhado para esse caso.
* Teste de tabuleiros exemplo, onde é testada a fiabilidade do valor returnado de um determinado critério para um dado tabuleiro.

In [23]:
boardAvgP = lambda game, state: alphabeta_cutoff_search_new(state, game, 5, eval_fn = decorator_eval_func_atk(boardAvg))
boardCombP = lambda game, state: alphabeta_cutoff_search_new(state, game, 5, eval_fn = decorator_eval_func_def(boardComb))
boardEmptyP = lambda game, state: alphabeta_cutoff_search_new(state, game, 5, eval_fn = decorator_eval_func_atk(boardEmpty))
boardPosP = lambda game, state: alphabeta_cutoff_search_new(state, game, 5, eval_fn = decorator_eval_func_def(boardPos))


board1 = [[2,2,2,4],[4,8,16,0],[64,0,2,0], [0,0,8,0]]
board2 = [[8,64,32,16],[8,64,16,8],[2,4,16,1024],[2,2,2,4]]
board3 = [[2,2,2,0],[4,8,0,0],[64,512,2,0], [64,2,8,0]]
board4 = [[2,2,2,2],[2,2,2,2],[2,2,2,2], [2,2,2,2]]
board5 = [[4,0,0,0],[0,0,0,0],[0,0,0,0], [0,0,0,0]]

all_boards = [board1,board2,board3,board4,board5]
all_states = list( map(lambda b: Jogo2048State(to_move="",utility=0,board=b,moves=0), all_boards) )


print(str(decorator_eval_func_atk(boardAvg)(all_states[5-1], "") == 1))
print(str(decorator_eval_func_atk(boardEmpty)(all_states[4-1], "") == 0))
print(str(decorator_eval_func_atk(boardEmpty)(all_states[5-1], "") == 1))
print(str(decorator_eval_func_atk(boardComb)(all_states[4-1], "") == 1))
print(str(decorator_eval_func_atk(boardPos)(all_states[5-1], "") == 1))
print(str(decorator_eval_func_atk(boardPos)(all_states[4-1], "") == 1))


print(decorator_eval_func_atk(boardAvg)(all_states[1-1], ""))
print(decorator_eval_func_atk(boardAvg)(all_states[2-1], ""))
print(decorator_eval_func_atk(boardAvg)(all_states[3-1], ""))
print(decorator_eval_func_atk(boardComb)(all_states[1-1], ""))
print(decorator_eval_func_atk(boardComb)(all_states[2-1], ""))
print(decorator_eval_func_atk(boardComb)(all_states[3-1], ""))
print(decorator_eval_func_atk(boardPos)(all_states[1-1], ""))
print(decorator_eval_func_atk(boardPos)(all_states[2-1], ""))
print(decorator_eval_func_atk(boardPos)(all_states[3-1], ""))
print(decorator_eval_func_atk(boardEmpty)(all_states[1-1], ""))
print(decorator_eval_func_atk(boardEmpty)(all_states[2-1], ""))
print(decorator_eval_func_atk(boardEmpty)(all_states[3-1], ""))

True
True
True
True
True
True
0.175
0.07763671875
0.11896306818181818
0.125
0.25
0.2916666666666667
0.598780487804878
0.4123456790123457
0.4247975318164288
0.4
0.0
0.3333333333333333


## Exemplos de jogos entre alguns desses jogadores e o Hipólito

In [24]:
game.jogar(atacante_hipolito.alg, createOptPlayer("player1", (83.1, 23.6, 35.5, 24.9))["player"].alg)
game.jogar(createOptPlayer("player1", (18.3, 23.0, 58.0, 72.4))["player"].alg, defensor_hipolito.alg)
game.jogar(boardEmptyP, defensor_hipolito.alg)
game.jogar(atacante_hipolito.alg, boardAvgP)

    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     4 
    0     0     0     2 
    0     0     0     0 
    0     0     0     0 
    0     0     0     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     2 
    0     0     0     4 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     0     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     2 
    0     0     2     4 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     2     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     2 
    0     0     4     4 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     4     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     4 
    0     0     0     8 


3736

## Exemplos de jogos entre dois dos vários jogadores desenvolvidos

In [25]:

game.jogar(atacante_hipolito.alg, createOptPlayer("player1", (83.1, 23.6, 35.5, 24.9))["player"].alg)
game.jogar(createOptPlayer("player1", (18.3, 23.0, 58.0, 72.4))["player"].alg, defensor_obsessivo.alg)
game.jogar(boardAvgP, boardEmptyP)
game.jogar(boardCombP, boardPosP)

    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     0     0 
    0     0     0     0 
    0     0     0     0 
    0     0     0     4 
    0     0     0     2 
    0     0     0     0 
    0     0     0     0 
    0     0     0     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     2 
    0     0     0     4 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     0     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     2 
    0     0     2     4 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     2     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     2 
    0     0     4     4 
    0     0     0     0 
    0     0     0     0 
    0     0     2     2 
    0     0     4     4 
    0     0     0     0 
    0     0     0     0 
    0     0     0     4 
    0     0     0     8 


172

## Processo de selecção dos jogadores para o torneio


<img src=".\imagens_2048\ormwphixo7y21.jpg" alt="Drawing" style="width: 350px;"/>


O processo de selecção teve por base a elaboração de um algoritmo genético. Após definir a base da função de avaliação a ser utilizada, que é constituída por quatro critérios definidos previamente (cada um a sua própria função de avaliação), multiplicados por um peso arbitrário, gerou-se funções de avaliação com um tuplo de quatro pesos aleatórios, correspondentes aos quatro critérios. 
<br>
O algoritmo genético corrido encarregou-se portanto de selecionar os pesos ideais para cada critério de forma a obter a melhor função de avaliação possível, utilizando torneios entre os jogadores de forma a determinar quais obtinham melhores resultados, registando um número arbitrário de melhores jogadores em ficheiros .txt, um correspondente ao ataque, e outro correspondente à defesa, de forma a poder utilizar esses valores para testes futuros.
Corremos este algoritmo genético em várias instâncias, para diferentes profundidades, com quatro variações, com o seguimento explicado abaixo:

* **Atacantes gerados vs Defesas gerados** - Em 1º lugar corremos várias instâncias de torneios entre jogadores gerados pelo algoritmo genético, tanto para o ataque como para a defesa, até os valores dos pesos convergirem na sua maioria.
* **Atacantes gerados / Atacantes hipólitos vs Defesas hipólitos / Defesas gerados** - em 2º lugar corremos várias instâncias de torneios entre atacantes gerados e defesas hipólitos, e atacantes hipólitos e defesas gerados, respetivamente, de forma a obter os atacantes e defesas gerados pelo algoritmo genético que obtinham melhores resultados contra os jogadores hipólitos.
* **Atacantes gerados / Atacantes obssessivos vs Defesas obsessivos/ Defesas gerados** - em 3º lugar, aplicamos o mesmo conceito da variação anterior, mas substituimos os jogadores hipólitos por jogadores obsessivos.
* **Atacantes "ótimos" gerados vs Defesas "ótimos" gerados** - Por último, corremos várias instâncias de torneios entre os melhores atacantes e defesas gerados pelo nosso algoritmo genético obtido das rondas anteriores, de forma a chegarmos ao atacante e defesa "ideais" para serem utilizados no torneio.

Como exemplo, podem observar o código de teste utilizado para correr a última variação / ronda dos nossos testes, tendo em conta que este código era muito semelhante ao utilizado para correr as outras variações dos nossos testes, sofrendo apenas alterações mínimas que afetavam a profundidade (depth), o numero de jogadores por torneio, o numero de jogadores eliminado e gerado ao fim de cada torneio/geração, entre outros:

```python
listAtk = []
listDef = []


init_pop = 0
num_gen = 1000
num_reproduce = 1
num_survivors = 3


for i in range(init_pop):
    ga = generate()
    listAtk.append( createPlayer( "Atk-", ga) )
    gd = generate()
    listDef.append( createPlayer( "Def-", gd) )

listAtk.append(createOptPlayer("Opt1A", (27.1, 72.5, 70.5, 75.0)))
listAtk.append(createOptPlayer("Opt2A", (99, 28, 78, 23)))
listAtk.append(createOptPlayer("Opt3A", (87.0, 67.5, 45.5, 75.0)))
listAtk.append(createOptPlayer("Opt4A", (18.299999999999997, 23.0, 58.0, 72.4)))

listDef.append(createOptPlayer("Opt1D", (83.1, 23.6, 35.5, 24.9)))
listDef.append(createOptPlayer("Opt2D", (24.799999999999997, 36.199999999999996, 87.5, 55.8)))
listDef.append(createOptPlayer("Opt3D", (68.6, 68.5, 8, 80.3)))
listDef.append(createOptPlayer("Opt4D", (54.3, 74.6, 52.9, 48.0)))


for g in range(num_gen):
    print("Generation: "+str(g))
    for j in range(len(listAtk)):
        listAtk[j]["score"] = 0
        listDef[j]["score"] = 0
    lists = faz_campeonato(listAtk, listDef)
    lists = fitness(lists, num_survivors)
    writetxt(listAtk, 0)
    writetxt(listDef, 1)
    listAtk = lists[0]
    listDef = lists[1]
    newAtk = []
    newDef = []
    for i in range(num_reproduce):
        ga = mutate( reproduce(listAtk[randint(0, len(listAtk)-1)]["adn"], listAtk[randint(0, len(listAtk)-1)]["adn"] ), g)
        newAtk.append( createPlayer( "Atk("+str(g)+")-", ga) )
        gd = mutate( reproduce(listDef[randint(0, len(listDef)-1)]["adn"], listDef[randint(0, len(listDef)-1)]["adn"] ), g )
        newDef.append( createPlayer( "Def("+str(g)+")-", gd) )
    listAtk.extend(newAtk)
    listDef.extend(newDef)
```

Dos quais obtemos os seguintes jogadores:
```
("OptOptA", (10.299999999999997, 27.0, 85.0, 74.5)
("OptOptD", (99.1, 74.6, 37.0, 57.900000000000006)
```

Sendo que os valores destes tuplos correspondem ao nome do jogador, e aos pesos dos critérios de avaliação, respetivamente.